<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comandos Basicos de Linux

In [0]:
!pwd # Obtener ruta de trabajo
!ls # Listado de archivos y carpetas
cd # Cambiar directorio de trabajo
!cp # Copiar un archivo

# Conectar rapidamente Google Earth Engine

Para conectarte con GEE necesitas superar dos pasos: autentificarte e inicializar. Con este pequeño snippet puedes saltarte el primer paso. Pero ten cuidado!

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"ESCRIBE_TU_TOKEN_AQUI"
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

# Crear mapa interactivo leaflet
Python no tiene un mapa por defecto como el code editor, sin embargo, podemos construir uno usando folium!

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# Monitoreo de tareas

Monitorea tus tareas en Google Earth Engine

In [0]:
import time 
def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)

# Funcion para enmascarar nubes SENTINEL-2

Enmascara nubes utilizando Sentinel-2

In [0]:
def maskS2clouds(image):
  qa = image.select('QA60')
  opaque_cloud = 1 << 10
  cirrus_cloud = 1 << 11
  mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
           .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
  clean_image = image.updateMask(mask)
  return clean_image

# Generar Base de Datos Espacial -I

In [0]:
def spatial_dataset(geom, prefix, folder, scale, variables = None):
  '''
  Genere una base de datos espacial. Para cualquier parte del Mundo usando 
  Google Earth Engine.
  
  > ARGUMENTOS:

  geom: area de interes (ee.Geometry.Polygon)
  prefix: homogenizar nombres (prefijo)
  folder: Nombre de la carpeta en Google Drive a guardar.
  scale: El tamaño de celda de la base de datos.
  variables: Variables soportadas en la funcion. Ver detalles:

  > DETALLES:

  SRTM: Modelo de Elevacion hidrologicamente condicionado. 
        Lehner, B., Verdin, K., Jarvis, A. (2008): New global hydrography 
        derived from spaceborne elevation data. Eos, Transactions, AGU, 
        89(10): 93-94.

  SLOPE: Mapa de pendiente generado a partir de MDE.

  ASPECT: Mapa de aspecto generado a partir de MDE.

  HILLSHADE: Modelo de sombras generado a partir de MDE.

  FLOWDIR: Direccion de flujo generado a partir de los datos
           SRTM.
  
  FLOWACC: Acumulacion de flujo generado a partir de los datos
           SRTM.

  ORGANIC: Materia organica (g/kg) al raz del suelo.
           Tomislav Hengl, & Ichsani Wheeler. (2018). Soil organic carbo
           n content in x 5 g / kg at 6 standard depths (0, 10, 30, 60,
           100 and 200 cm) at 250 m resolution (Version v02) [Data set].
            Zenodo. 10.5281/zenodo.1475457

  DENSITY: Densidad aparente del suelo al raz del suelo.

  RAIN: Datos de lluvia mensual (mm/month). 
        Abatzoglou, J.T., S.Z. Dobrowski, S.A. Parks, K.C. Hegewisch, 2018,
        Terraclimate, a high-resolution global dataset of monthly climate 
        and climatic water balance from 1958-2015, Scientific Data 5:170191,
         doi: 10.1038/sdata.2017.191

  ETP: Datos de evapotranspiracion de referencia (ASCE Penman-Montieth).

  NDVI_mean: Datos de NVDI calculado a partir de datos Landsat.

  LANDUSE: Uso de suelo Copernicus.
           Buchhorn, M. ; Smets, B. ; Bertels, L. ; Lesiv, M. ; Tsendbazar,
           N. - E. ; Herold, M. ; Fritz, S. Copernicus Global Land Service:
           Land Cover 100m: epoch 2015: Globe. Dataset of the global component
           of the Copernicus Land Monitoring Service 2019. DOI 
           10.5281/zenodo.3243509
  '''
  if variables is None:
    variables = ['srtm','slope','aspect','hillshade','flowdir','flowacc',
                 'organic','density','rain','etp','ndvi_mean','landcover']

  ee_var = {}
  
  # 1. ee.Image relacionadas a un key
  #    e.g: srtm ---> WWF/HydroSHEDS/03CONDEM
  if 'srtm' in variables:
    srtm = ee.Image("WWF/HydroSHEDS/03CONDEM")
    ee_var['srtm'] = srtm    
  if 'slope' in variables:
    slope = ee.Terrain.slope(srtm)
    ee_var['slope'] = slope
  if 'aspect' in variables:
    aspect = ee.Terrain.aspect(srtm)
    ee_var['aspect'] = aspect    
  if 'hillshade' in variables:
    hillshade = ee.Terrain.hillshade(srtm)
    ee_var['hillshade'] = hillshade
  if 'flowdir' in variables:   
    flowdir = ee.Image("WWF/HydroSHEDS/03DIR")
    ee_var['flowdir'] = flowdir
  if 'flowacc' in variables:   
    flowacc = ee.Image("WWF/HydroSHEDS/15ACC")
    ee_var['flowacc'] = flowacc
  if 'organic' in variables:       
    organic = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")
    ee_var['organic'] = organic
  if 'density' in variables:       
    density = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")
    ee_var['density'] = density
  if 'rain' in variables:           
    rain = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").mean().select('pr')
    ee_var['rain'] = rain
  if 'etp' in variables:           
    etp = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").mean().select('pet')    
    ee_var['etp'] = etp
  if 'ndvi_mean' in variables:               
    ndvi_mean = ee.ImageCollection("LANDSAT/LT05/C01/T1_32DAY_NDVI").mean().select('NDVI')
    ee_var['ndvi_mean'] = ndvi_mean
  if 'landcover' in variables:                   
    landcover = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").first().select('discrete_classification')
    ee_var['landcover'] = landcover

  # 2. Descargar imagenes 
  for key, value in ee_var.items():
    to_download = ee.Image(value).clip(geom)
    task = ee.batch.Export.image.toDrive(
      image=to_download,
      description= prefix+key,
      folder=folder,      
      scale=scale,
      region = geom
    ) 
    task.start()
    print('Descargando: '+prefix+key+'.tif')

# Generador de ImageCollection Sentinel2

In [0]:
def clean_s2(init_date, last_date, roi, cloud_per, calendar, bands):
  def maskS2clouds(image):
    qa = image.select('QA60')
    opaque_cloud = 1 << 10 #1*2**10
    cirrus_cloud = 1 << 11 #1*2**11
    mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
             .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    clean_image = image.updateMask(mask).select(bands)
    return clean_image.copyProperties(image, ['system:time_start'])
  ic_s2 = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(roi)\
               .filterDate(init_date, last_date)\
               .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_per))\
               .map(maskS2clouds)
  return ic_s2

# Obtener fechas en un  ImageCollection

In [0]:
from datetime import datetime as dt
def ee_get_dates(ic):
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))

# Descargar un ImageCollection

In [0]:
from datetime import datetime
import time 

def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)    
    
def ee_download_images(ic, region, prefix, folder, scale):
  nimages = ic.size().getInfo()
  ic_list = ic.toList(nimages)  
  for n_img in range(nimages):
    to_download = ee.Image(ic_list.get(n_img))
    timestamp = to_download.get('system:time_start').getInfo()
    dt_object = datetime.fromtimestamp(timestamp/1000).strftime('%Y_%m_%d_%H_%M_%S')
    task = ee.batch.Export.image.toDrive(
      image=to_download,
      description= prefix+dt_object,
      folder=folder,     
      scale=scale,
      region = region
    )
    print('Descargando: '+prefix+dt_object+'.tif')
    task.start()
    ee_monitoring(task)

# Generar animacion - NDVI MODIS

In [0]:
def monthly_ndvi_animation(area, vizparams = None, framesPerSecond = 10):
  if vizparams is None:
    vizparams = {'min' : 0.0,
                 'max' : 9000.0,
                 'palette' : ['#051852', '#FFFFFF', '#C7B59B', '#A8B255',
                              '#A3C020', '#76AD00', '#429001', '#006400',
                              '#003B00', '#000000']
                 }
  # Cargamos los datos de NDVI de MODIS
  col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

  # Agregamos la propiedad day-of-year (DOY) para cada imagen
  def add_day(img):
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)
  
  col = col.map(add_day)

  # Obtenga una colección de imágenes distintas de 'doy'.
  distinctDOY = col.filterDate('2013-01-01', '2014-01-01')

  # Defina un filtro que identifique que imagenes de la 'coleccion completa'
  # coincide con el DOY de la colección de referencia.
  my_filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')

  # Define el Join 
  join = ee.Join.saveAll('doy_matches')

  # Aplicar el join y convertir el FeatureCollection resultante en un ImageCollection.
  joinCol = ee.ImageCollection(join.apply(distinctDOY, col, my_filter))

  # Aplicar la mediana entre las colecciones DOY coincidentes.
  def reduce_each_image(img):
    doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return doyCol.reduce(ee.Reducer.median())

  comp = joinCol.map(reduce_each_image)

  # Cree imagenes de visualizacion RGB para usar como cuadros de animacion.
  rgbVis = comp.map(lambda x:x.visualize(**vizparams).clip(area))

  # Definir argumentos de visualización GIF.
  gifParams = {
    'region': area.bounds(),
    'dimensions': 600,
    'crs': 'EPSG:3857',
    'framesPerSecond': framesPerSecond,
    'format': 'gif'
  }
  # Imprima la URL GIF en la consola.
  return rgbVis.getVideoThumbURL(gifParams)